In [1]:
!ls

In [6]:
import pandas as pd
import numpy as np
from collections import defaultdict


table_gt = pd.read_csv('tables_per_page.txt', header=None)
table_gt.columns = columns=['page','tables','end','start']
table_gt['slice'] = [list(range(int(s),int(e+1))) if not np.isnan(s) else [int(e)] for s,e in table_gt[['start','end']].values]

table_gt

,page,tables,end,start,slice
0,1,5,2013,2009.0,"[2009, 2010, 2011, 2012, 2013]"
1,2,5,2008,2004.0,"[2004, 2005, 2006, 2007, 2008]"
2,3,5,2003,1999.0,"[1999, 2000, 2001, 2002, 2003]"
3,4,5,1998,1994.0,"[1994, 1995, 1996, 1997, 1998]"
4,5,5,1993,1989.0,"[1989, 1990, 1991, 1992, 1993]"
...,...,...,...,...,...
63,64,1,1918,NaN,[1918]
64,65,2,1917,1916.0,"[1916, 1917]"
65,66,7,1915,1872.0,"[1872, 1873, 1874, 1875, 1876, 1877, 1878, 187..."
66,67,7,1871,1865.0,"[1865, 1866, 1867, 1868, 1869, 1870, 1871]"


In [7]:
from tqdm.auto import tqdm
import tabula



pdf = 'fed_individual_rate_history_nominal.pdf'
pdf = 'https://files.taxfoundation.org/legacy/docs/fed_individual_rate_history_nominal.pdf'

areas = {
    'Jointly': [0, 0, 100, 28],
    'Separately': [0, 28, 100, 49],
    'Single': [0, 52, 100, 75],
    'Head': [0, 75, 100, 100],
}

page_tables = {k: defaultdict(list) for k in areas.keys()}


for table_key, area in areas.items():

    dfs = []
    for i in tqdm(range(1,69)):
        
        df = tabula.read_pdf(
            pdf, 
            pages=i, 
            multiple_tables=True,
            stream=True,
            guess=True,
            area=area,
            relative_area=True,
        )
        dfs.append(df)


    for page_n, page in enumerate(dfs):
        d = pd.concat(page)
        d = pd.DataFrame([" ".join(x) for x in d.values.astype(str)])

        d.columns = ['rows']

        d = d[(d['rows'].str.contains('%')) | (d['rows'].str.contains(table_key))]
        d['table'] = (d['rows'].str.contains(table_key)).cumsum()

        for g, gdf in d.groupby('table'):
            data = gdf.iloc[1:,0].values.astype(str)
            table = []
            for d in data:
#                 d = d.split()

                d = d.replace('-','').replace(',','').replace('%','').replace('$','').split()
                vals = [None] * 3
                vals[:len(d)] = d

                table.append(vals)

            table = pd.DataFrame(table)

            page_tables[table_key][page_n].append(table)

In [8]:
total_data = []

for tax_type in page_tables.keys():
    for page_number, page_dataframes in page_tables[tax_type].items():
#         print(
#             page_number, 
#             len(page_dataframes), 
#             table_gt[table_gt['page'] == page_number+1]['slice'],
#         )
        for tmp_pg_df, year in zip(page_dataframes, table_gt[table_gt['page'] == page_number+1]['slice'].values[0]):
            tmp_pg_df['year'] = year
            tmp_pg_df['tax_type'] = tax_type
            tmp_pg_df['page_num'] = page_number
            
            total_data.append(tmp_pg_df)
            
total_data = pd.concat(total_data)
total_data

,0,1,2,year,tax_type,page_num,3,4,5,6,7
0,10.0,0,17850,2009,Jointly,0,NaN,NaN,NaN,NaN,NaN
1,15.0,17850,72500,2009,Jointly,0,NaN,NaN,NaN,NaN,NaN
2,25.0,72500,146400,2009,Jointly,0,NaN,NaN,NaN,NaN,NaN
3,28.0,146400,223050,2009,Jointly,0,NaN,NaN,NaN,NaN,NaN
4,33.0,223050,398350,2009,Jointly,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20,81.0,90000,100000,1952,Head,30,NaN,NaN,NaN,NaN,NaN
21,85.0,100000,150000,1952,Head,30,NaN,NaN,NaN,NaN,NaN
22,88.0,150000,200000,1952,Head,30,NaN,NaN,NaN,NaN,NaN
23,91.0,200000,300000,1952,Head,30,NaN,NaN,NaN,NaN,NaN


In [9]:
with pd.option_context("display.max_rows", 1300):
    display(total_data[~total_data.iloc[:,6].isna()])

,0,1,2,year,tax_type,page_num,3,4,5,6,7
2,(a),A,33,1992,Jointly,4,"""rate","bubble""",applied,between,7840
2,(a),A,33,1993,Jointly,4,"""rate","bubble""",applied,between,7485
2,(a),A,33,1986,Jointly,5,"""rate","bubble""",applied,between,7190
50,bottom,rates,as,1923,Jointly,58,3,and,43.5,at,http://www.irs.
0,10.0,0,802,2004,Separately,1,nan,NaN,NaN,NaN,NaN
1,15.0,8025,3255,2004,Separately,1,nan,NaN,NaN,NaN,NaN
2,25.0,32550,6572,2004,Separately,1,nan,NaN,NaN,NaN,NaN
3,28.0,65725,10015,2004,Separately,1,nan,NaN,NaN,NaN,NaN
4,33.0,100150,17885,2004,Separately,1,nan,NaN,NaN,NaN,NaN
0,10.0,0,782,2005,Separately,1,nan,NaN,NaN,NaN,NaN


In [10]:
numeric_perc = []
for row_perc in total_data.iloc[:,0].values:
    try:
        float(row_perc)
        numeric_perc.append(True)
    except:
        numeric_perc.append(False)
        
print(sum(numeric_perc), len(total_data))
        
total_data[~total_data.iloc[:,6].isna()]

4462 4469


,0,1,2,year,tax_type,page_num,3,4,5,6,7
2,(a),A,33,1992,Jointly,4,"""rate","bubble""",applied,between,7840
2,(a),A,33,1993,Jointly,4,"""rate","bubble""",applied,between,7485
2,(a),A,33,1986,Jointly,5,"""rate","bubble""",applied,between,7190
50,bottom,rates,as,1923,Jointly,58,3,and,43.5,at,http://www.irs.
0,10.0,0,802,2004,Separately,1,nan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21,68.0,80000,90000,1971,Single,16,nan,NaN,NaN,NaN,NaN
22,69.0,90000,100000,1971,Single,16,nan,NaN,NaN,NaN,NaN
2,d,saved,by,1992,Head,4,applying,the,15,rate.,NaN
2,d,saved,by,1993,Head,4,applying,the,15,rate.,NaN


In [11]:

for g, thing in total_data[~total_data.iloc[:,6].isna()].groupby(['tax_type','page_num']):
    print(g)
    display(thing)

('Head', 4)


,0,1,2,year,tax_type,page_num,3,4,5,6,7
2,d,saved,by,1992,Head,4,applying,the,15,rate.,NaN
2,d,saved,by,1993,Head,4,applying,the,15,rate.,NaN


('Head', 5)


,0,1,2,year,tax_type,page_num,3,4,5,6,7
1,d,saved,by,1986,Head,5,applying,the,15,rate.,NaN


('Jointly', 4)


...

,0,1,2,year,tax_type,page_num,3,4,5,6,7
0,15.0,500,1000,1973,Single,14,nan,NaN,NaN,NaN,NaN
1,16.0,1000,1500,1973,Single,14,nan,NaN,NaN,NaN,NaN
2,17.0,1500,2000,1973,Single,14,nan,NaN,NaN,NaN,NaN
3,19.0,2000,4000,1973,Single,14,nan,NaN,NaN,NaN,NaN
4,21.0,4000,6000,1973,Single,14,nan,NaN,NaN,NaN,NaN
5,24.0,6000,8000,1973,Single,14,nan,NaN,NaN,NaN,NaN
6,25.0,8000,10000,1973,Single,14,nan,NaN,NaN,NaN,NaN
7,27.0,10000,12000,1973,Single,14,nan,NaN,NaN,NaN,NaN
8,29.0,12000,14000,1973,Single,14,nan,NaN,NaN,NaN,NaN
9,31.0,14000,16000,1973,Single,14,nan,NaN,NaN,NaN,NaN


('Single', 15)


,0,1,2,year,tax_type,page_num,3,4,5,6,7
0,15.0,500,1000,1972,Single,15,nan,NaN,NaN,NaN,NaN
1,16.0,1000,1500,1972,Single,15,nan,NaN,NaN,NaN,NaN
2,17.0,1500,2000,1972,Single,15,nan,NaN,NaN,NaN,NaN
3,19.0,2000,4000,1972,Single,15,nan,NaN,NaN,NaN,NaN
4,21.0,4000,6000,1972,Single,15,nan,NaN,NaN,NaN,NaN
5,24.0,6000,8000,1972,Single,15,nan,NaN,NaN,NaN,NaN
6,25.0,8000,10000,1972,Single,15,nan,NaN,NaN,NaN,NaN
7,27.0,10000,12000,1972,Single,15,nan,NaN,NaN,NaN,NaN
8,29.0,12000,14000,1972,Single,15,nan,NaN,NaN,NaN,NaN
9,31.0,14000,16000,1972,Single,15,nan,NaN,NaN,NaN,NaN


('Single', 16)


,0,1,2,year,tax_type,page_num,3,4,5,6,7
0,15.0,500,1000,1971,Single,16,nan,NaN,NaN,NaN,NaN
1,16.0,1000,1500,1971,Single,16,nan,NaN,NaN,NaN,NaN
2,17.0,1500,2000,1971,Single,16,nan,NaN,NaN,NaN,NaN
3,19.0,2000,4000,1971,Single,16,nan,NaN,NaN,NaN,NaN
4,21.0,4000,6000,1971,Single,16,nan,NaN,NaN,NaN,NaN
5,24.0,6000,8000,1971,Single,16,nan,NaN,NaN,NaN,NaN
6,25.0,8000,10000,1971,Single,16,nan,NaN,NaN,NaN,NaN
7,27.0,10000,12000,1971,Single,16,nan,NaN,NaN,NaN,NaN
8,29.0,12000,14000,1971,Single,16,nan,NaN,NaN,NaN,NaN
9,31.0,14000,16000,1971,Single,16,nan,NaN,NaN,NaN,NaN


In [12]:
total_data[numeric_perc].iloc[:,:6].to_csv('fed_individual_rate_history_nominal_2.csv')